In [1]:
import os
import json
from difflib import unified_diff
import nltk
#nltk.download('punkt')
from nltk import word_tokenize


cwd = os.getcwd()
print(cwd)

C:\Users\u0115374\Documents\PhD\Courses\Big Data\SparkStream


In [ ]:
def make_diff(old, new):
    return '\n'.join([ l for l in unified_diff(old.split('\n'), new.split('\n')) if l.startswith('+') or l.startswith('-') ])

In [ ]:
'''Function to extract the added pieces of text in the document.'''
def get_added_text(diff):
    edits = []
    for text in diff.split('\n'):
        if text.startswith("+"):
            edits.append(text)
    #print(edits)        
    return edits